# Import data from Google Cloud Storage (GCS)

It's convenient and secure to host data in the cloud for data labeling, then sync task references to Label Studio to allow data annotators to view and label the tasks without your data leaving the secure cloud bucket. 

If your data is hosted in Google Cloud Storage (GCS), you can write a Python script to continuously sync data from the bucket with Label Studio. Follow this example to see how to do that with the [Label Studio SDK](https://labelstud.io/sdk/index.html). 

## Connect to your GCS bucket

Connect to your GCS bucket and create a list of task references that Label Studio can use, based on the contents of your bucket. 

In [ ]:
!pip install --upgrade google-api-python-client

In [ ]:
import os
from google.cloud import storage as google_storage

BUCKET_NAME = 'my-bucket'  # specify your bucket name here
GOOGLE_APPLICATION_CREDENTIALS = 'my-service-account-credentials.json'  # specify your GCS credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_APPLICATION_CREDENTIALS

google_client = google_storage.Client()
bucket = google_client.get_bucket(BUCKET_NAME)
tasks = []
for filename in bucket.list_blobs():
    tasks.append({'image': f'gs://{BUCKET_NAME}/{filename}'})

## Create a Label Studio Project

Connect to the Label Studio API with your personal API key, which you can retrieve from your user account page, and confirm you can successfully connect:

In [ ]:
from label_studio_sdk import Client
LABEL_STUDIO_URL = 'http://localhost:8000'
API_KEY = '91b3b61589784ed069b138eae3d5a5fe1e909f57'

ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
ls.check_connection()

Create the project. In this example, the project is a basic [image object detection project](https://labelstud.io/templates/image_bbox.html):

In [ ]:
project = ls.start_project(
    title='Image Annotation Project from SDK',
    label_config='''
    <View>
        <Image name="image" value="$image"/>
        <RectangleLabels name="objects" toName="image">
            <Choice value="Airplane"/>
            <Choice value="Car"/>
        </RectangleLabels>
    </View>
    '''
)

## Connect to your GCS bucket

Connect your newly-created project to your GCS bucket:

In [ ]:
project.connect_google_import_storage(
    bucket=BUCKET_NAME,
    google_application_credentials=GOOGLE_APPLICATION_CREDENTIALS
)

## Sync tasks from GCS to Label Studio

After connecting to your bucket, you can import your private GCS links to Label Studio. When opening in Label Studio interface, they're automatically presigned for security! 

In [ ]:
project.import_tasks(tasks)

## Conclusion

In a few lines of code you assessed the data in your bucket, set up a new labeling project, and synced the tasks to the project. You can adapt this example to more easily create a data creation to data labeling pipeline.